Connect client


In [29]:
# !pip3 install session-info
# import session_info
# session_info.show()

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct
import numpy as np
import random
import pandas as pd
import time, pickle

import utils

In [6]:
qdrantClient = QdrantClient(host='localhost', port=6333, timeout=10000000)
# base_vectors = utils.read_fvecs("../../dataset/siftsmall/siftsmall_base.fvecs")
# query_vectors = utils.read_fvecs("../../dataset/siftsmall/siftsmall_query.fvecs")
# knn_groundtruth = utils.read_ivecs("../../dataset/siftsmall/siftsmall_groundtruth.ivecs")
# base_vectors = utils.read_fvecs("../../dataset/sift/sift_base.fvecs")
# query_vectors = utils.read_fvecs("../../dataset/sift/sift_query.fvecs")
# knn_groundtruth = utils.read_ivecs("../../dataset/sift/sift_groundtruth.ivecs")

with open('query_vectors_with_attributes.pkl', 'rb') as f:
    query_vectors_with_attributes = pickle.load(f)

with open('base_vectors_with_attributes.pkl', 'rb') as f:
    base_vectors_with_attributes = pickle.load(f)

with open('calculated_truth.pkl', 'rb') as f:
    truth = pickle.load(f)

In [14]:
len(base_vectors_with_attributes)

1000000

# Attribute filtering 

Add random boolean attributes to base vectors

Create qdrant collection

In [12]:
vector_size = 128
collection_name = "attribute_filtering_1M"

qdrantClient.delete_collection(collection_name=collection_name)

qdrantClient.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.EUCLID),
)

True

In [13]:
start_time = time.time()
batch_points = [PointStruct(id=i, vector=elem["vector"], payload= {"attr1": elem["attr1"], "attr2": elem["attr2"], "attr3": elem["attr3"]}) for i, elem in base_vectors_with_attributes.iterrows()]
end_time = time.time()
time_span = end_time - start_time
print(f'Points created in {time_span}')

Points created in 41.344099283218384


In [32]:
base_vectors_with_attributes.iloc[0]

vector    [0.0, 16.0, 35.0, 5.0, 32.0, 31.0, 14.0, 10.0,...
attr1                                                  True
attr2                                                  True
attr3                                                 False
Name: 0, dtype: object

In [33]:
batch_points[0]

PointStruct(id=0, vector=[0.0, 16.0, 35.0, 5.0, 32.0, 31.0, 14.0, 10.0, 11.0, 78.0, 55.0, 10.0, 45.0, 83.0, 11.0, 6.0, 14.0, 57.0, 102.0, 75.0, 20.0, 8.0, 3.0, 5.0, 67.0, 17.0, 19.0, 26.0, 5.0, 0.0, 1.0, 22.0, 60.0, 26.0, 7.0, 1.0, 18.0, 22.0, 84.0, 53.0, 85.0, 119.0, 119.0, 4.0, 24.0, 18.0, 7.0, 7.0, 1.0, 81.0, 106.0, 102.0, 72.0, 30.0, 6.0, 0.0, 9.0, 1.0, 9.0, 119.0, 72.0, 1.0, 4.0, 33.0, 119.0, 29.0, 6.0, 1.0, 0.0, 1.0, 14.0, 52.0, 119.0, 30.0, 3.0, 0.0, 0.0, 55.0, 92.0, 111.0, 2.0, 5.0, 4.0, 9.0, 22.0, 89.0, 96.0, 14.0, 1.0, 0.0, 1.0, 82.0, 59.0, 16.0, 20.0, 5.0, 25.0, 14.0, 11.0, 4.0, 0.0, 0.0, 1.0, 26.0, 47.0, 23.0, 4.0, 0.0, 0.0, 4.0, 38.0, 83.0, 30.0, 14.0, 9.0, 4.0, 9.0, 17.0, 23.0, 41.0, 0.0, 0.0, 2.0, 8.0, 19.0, 25.0, 23.0, 1.0], payload={'attr1': True, 'attr2': True, 'attr3': False})

Create point-structure elements and insert them in DB (see qdrant docs)

In [15]:
batch_size = 50000
num_batches = len(base_vectors_with_attributes) // batch_size + int(len(base_vectors_with_attributes) % batch_size > 0)
print(f'Number of batches: {num_batches}')

for batch_idx in range(num_batches):
    print(f'Current progress: {(batch_idx+1)*batch_size}/{len(base_vectors_with_attributes)}', end='\r')
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(base_vectors_with_attributes))

    batch_points_i = batch_points[start_idx:end_idx]

    operation_info = qdrantClient.upsert(
        collection_name=collection_name,
        wait=True,
        points=batch_points_i
    )

Number of batches: 20


In [18]:
res = qdrantClient.get_collection(collection_name=collection_name)
# Wait until status is green

In [22]:
res.status == 'green'

True

Add random boolean attributes to query vectors

Now search the knn in qdrant

In [43]:
print(f'Search function starting')
start_time = time.time()

result_ids = []
for i,elem in query_vectors_with_attributes.iterrows():
    if i == 100:
        break
    print(f'{i}/{len(query_vectors_with_attributes)}', end='\r')
    # print(elem)
    vec = elem["vector"]
    attr1 = elem["attr1"]
    attr2 = elem["attr2"]
    attr3 = elem["attr3"]
    # print(attr1, attr2, attr3)
    search_result = qdrantClient.search(
        collection_name="attribute_filtering_1M", 
        query_vector=vec, 
        query_filter=models.Filter(
            # must = AND
            must=[
                models.FieldCondition(
                    key="attr1",
                    match=models.MatchValue(
                        value=attr1,
                    ),
                ),
                models.FieldCondition(
                    key="attr2",
                    match=models.MatchValue(
                        value=attr2,
                    ),
                ),
                models.FieldCondition(
                    key="attr3",
                    match=models.MatchValue(
                        value=attr3,
                    ),
                )
            ]
        ),
        limit=100,
        # search_params=models.SearchParams(
        #     exact=True,  # Turns on the exact search mode
        # ),
    )
    result_ids.append([elem.id for elem in search_result])

end_time = time.time()
time_span = end_time - start_time
print(f'Search function took {end_time - start_time} seconds')

Search function starting
Search function took 73.63646221160889 seconds


In [ ]:
result

In [36]:
truth[0]

array([721706,  49874, 701919, 904911, 556209, 806773, 465294, 374617,
       956733, 670103, 871600, 570366, 879793,  13612, 238549, 931855,
       561694, 708525, 392032, 977462, 875747, 758431, 244266, 956632,
        86760,  72761, 454389, 913505, 377461,  16155, 574941, 222238,
       731216, 711701, 570424, 820116, 377591, 710839, 403026, 705721,
       844361, 946913, 308335, 502709, 793624, 921077, 393275, 223825,
       605201, 561554, 698111, 973897, 753203, 964853, 380345,  19323,
       707152, 226910, 929411, 203309, 973399, 684790, 758178, 266275,
       158920, 586028, 868741,  14883, 868599, 843560, 408470, 845316,
       564454, 977640, 173255, 178211, 502632, 977398, 574956, 563588,
        87321,  98054,  89430, 932749,  14750, 707219, 407971, 982563,
       732612, 511003, 561446, 504491,  68461, 918776, 130227, 399570,
       831264, 486428,  98627, 885796])

In [39]:
np.array(result_ids[0])

array([721706,  49874, 701919, 904911, 806773, 465294, 956733, 871600,
       931855, 758431, 956632,  86760,  72761, 454389, 731216, 403026,
       705721, 844361, 502709, 793624, 393275, 605201, 561554, 973897,
       964853, 380345, 226910, 929411, 973399, 758178, 158920, 843560,
       502632,  98054, 932749, 707219, 407971, 732612, 511003, 561446,
       399570, 486428,  98627, 888278, 603674, 242295,  10035, 943041,
       516029, 394649, 883449, 164763, 398806, 390750, 253562, 909759,
       407565, 505815, 729530, 843429, 869739, 932200, 201315, 265593,
       489167, 861998, 686727, 255342, 870509, 265328, 724462, 919854,
       844812, 562084, 844918, 920028, 841933, 237902, 913922, 842548,
       590734,  60098, 683620, 965225, 664118, 887093, 863797, 842222,
       840983, 905416, 842861, 472271, 705078, 911485, 922981, 125539,
       946969, 993681, 924882, 732637])

In [47]:
sum = 0
for i in range(10):
    sum += len(np.intersect1d(truth[i], result_ids_exact[i]))
sum

1000

In [40]:
true_positives = 0
n_classified = 0
for i,elem in enumerate(result_ids):
    true_positives_iter = len(np.intersect1d(truth[i], elem))
    true_positives += true_positives_iter
    n_classified += len(elem)

print(f'QPS = {(len(query_vectors_with_attributes) / time_span):.4f}')
print(f'Average recall: {true_positives/n_classified}')

QPS = 53.3294
Average recall: 0.26672331261159776


# Graveyard

In [76]:
base_vectors_with_attributes.iloc[8021]

vector    [3.0, 0.0, 0.0, 0.0, 13.0, 25.0, 23.0, 22.0, 0...
attr1                                                 False
attr2                                                 False
attr3                                                 False
Name: 8021, dtype: object

In [ ]:
import numpy as np

res = [elem.id for elem in search_result]
res = np.array(res, dtype=np.int32)

intersection

In [ ]:
# for vec in query_vectors:
#     search_result = qdrantClient.search(
#         collection_name="test_collection", query_vector=[0.0 for _ in range(vector_size)], limit=100
#     )
#     break

for vec in query_vectors:
    search_result = qdrantClient.search(
        collection_name="test_collection", query_vector=vec, limit=100
    )
    break


In [ ]:
columns_to_match = base_vectors_with_attributes.columns[1:]
print(f"We will be matching the following (boolean) values: {query_vectors_with_attributes[columns_to_match].iloc[0]}")

# Create a boolean mask based on values in the 'r' row
# mask = (base_vectors_with_attributes[columns_to_match] == query_vectors_with_attributes[columns_to_match].iloc[0]).all(axis=1)

# filtered_df = base_vectors_with_attributes[mask]
# filtered_df

import utils
utils

In [ ]:
# query_vectors_with_attributes.columns
boolean_columns = query_vectors_with_attributes.columns[query_vectors_with_attributes.dtypes == bool].difference(['vector'])
mask = (base_vectors_with_attributes[boolean_columns] == query_vectors_with_attributes[boolean_columns].iloc[0]).all(axis=1)
mask